# Chaos polynomial : description des algorithmes

Dans cette page, nous décrivons les algorithmes principaux du chaos polynomial dans OpenTURNS.


## La classe FunctionalChaosAlgorithm

La méthode `run` de la classe `FunctionalChaosAlgorithm` réalise les étapes suivantes.
* Création de la fonction composée $Y = g(T^{-1}(\xi))$ à partir de la fonction $g$ et de la transformation iso-probabiliste $T$. Cela fait appel à la classe `ComposedFunction`. 
* Pour chaque composante du vecteur de sortie $Y$, l'algorithme calcule la décomposition de la composante. Le résultat principal de cette étape est la liste des indices des coefficients et la valeur des coefficients.

Pour une composante donnée du vecteur de sortie, l'algorithme est le suivant.
* Calcul de la base polynomiale initiale avec la méthode `computeInitialBasis` de la classe `AdaptiveStrategy`.
* Tant que
  * Calcul des coefficients associés à la stratégie de projection par appel à la méthode `computeCoefficients` de la classe `ProjectionStrategy`.
  * Si les résidus sont inférieurs au seuil, l'algorithme s'arrête.
  * Met à jour la base polynomiale avec la méthode `updateBasis` de la classe `AdaptiveStrategy`.
  * Si l'objet `adaptiveStrategy` n'a pas vu sa base polynomial ni augmentée, ni réduite, alors l'algorithme s'arrête.

## Variantes d'AdaptiveStrategy

Les implémentations d'AdaptiveStrategy ont pour objectif de générer des métamodèles de chaos polynomial creux, dans lesquels le nombre de coefficients est limité. L'intérêt de ces algorithmes est de pouvoir limiter l'effet du surapprentissage lorsque le nombre de coefficients augmente. Ils sont une alternative technique statistiquement moins fondée que la méthode de sélection de modèle LARS, qui est décrite dans la suite.

Trois implémentations d'`AdaptiveStragy` sont disponibles.
* Dans la classe `FixedStrategy`, l'utilisateur fixe un indice maximal de coefficient. 
  * Le calcul de la base initial par `computeInitialBasis` génère toute la base fonctionnelle jusqu'à un indice maximal donné par l'utilisateur à la création de l'objet. 
  * La méthode `updateBasis` n'a pas d'effet.
* Dans la classe `SequentialStrategy`, l'utilisateur fixe une taille maximale pour le nombre de coefficients dans la base polynomiale. 
  * Le calcul de la base initial par `computeInitialBasis` génère le premier élément de la base, d'indice 0. 
  * La méthode `updateBasis` fonctionne de la manière suivante.
    * Si la base contient déjà un nombre maximal de coefficients donné par l'utilisateur lors de la création de l'objet, la méthode ne fait rien.
    * Sinon, elle ajoute le coefficient dans la base. 
* Dans la classe `CleaningStrategy`, l'utilisateur fixe 
  * un nombre maximum de coefficients dans la base,
  * le nombre de termes retenus,
  * un seuil de significativité.
  * Le calcul de la base initial par `computeInitialBasis` génère tous les éléments jusqu'au minimum du nombre de coefficients maximum et du nombre de termes retenus.
  * La méthode `updateBasis` fonctionne sur la base d'un tri des coefficients par ordre décroissant en valeur absolue. L'algorithme peut ajouter ou retirer des éléments dans la base.
  


## Calcul des coefficients par ProjectionStrategy

La classe `ProjectionStrategy` possède deux implémentations pour estimer les coefficients.
* La classe `LeastSquaresStrategy` implémente les moindres carrés.
* La classe `IntegrationStrategy` implémente la méthode d'estimation par intégration.
